# Example parquet data source

We demonstrate using Intake to load dataframe data from parquet

In [1]:
# a simple catalog with a single source pointing to parquet data
%cat sample.yml

plugins:
  source:
    - module: intake_parquet
sources:
  - name: test
    description: Short example parquet data
    driver: parquet
    args:
      urlpath: !template '{{ CATALOG_DIR }}/test.parq'


In [2]:
# the test data is actually made of two data files and metadata
%ls test.parq/

_common_metadata  _metadata         part.0.parquet    part.1.parquet


We proceed in the usual way, by loading the sample catalog, which contains just one entry.

In [3]:
from intake.catalog import Catalog

In [4]:
cat = Catalog('sample.yml')

In [5]:
list(cat)

['test']

In [6]:
cat.test

{'user_parameters': [], 'direct_access': 'forbid', 'description': 'Short example parquet data', 'container': 'dataframe'}

In [7]:
source = cat.test.get()

Unlike many data source, for parquet we know a lot about the data before loading any of it. Notice that the complete shape is known, and for data which originated with pandas, there is also additional information about the original dataframe types and index (if any).

In [8]:
source.discover()

{'datashape': None,
 'dtype': {'bcat': 'category',
  'bhello': dtype('O'),
  'cat': 'category',
  'f': dtype('float64'),
  'hello': dtype('O'),
  'i32': dtype('int32'),
  'i64': dtype('int64')},
 'metadata': {'pandas': '{"columns": [{"metadata": null, "name": "bhello", "numpy_type": "object", "pandas_type": "bytes"}, {"metadata": null, "name": "f", "numpy_type": "float64", "pandas_type": "float64"}, {"metadata": null, "name": "i32", "numpy_type": "int32", "pandas_type": "int32"}, {"metadata": null, "name": "i64", "numpy_type": "int64", "pandas_type": "int64"}, {"metadata": null, "name": "hello", "numpy_type": "object", "pandas_type": "unicode"}, {"metadata": {"num_categories": 3, "ordered": false}, "name": "bcat", "numpy_type": "int8", "pandas_type": "categorical"}, {"metadata": {"num_categories": 3, "ordered": false}, "name": "cat", "numpy_type": "int8", "pandas_type": "categorical"}], "index_columns": [], "pandas_version": "0.20.1"}'},
 'npartitions': 2,
 'shape': (2002, 7)}

In [9]:
# we notice that some of the Object-type fields are actually categorical
# when loaded - this is good for saving memory.
source.read_partition(0).head()

,bhello,f,i32,i64,hello,bcat,cat
0,b'people',1.538369,-68805,-3279161491,people,b'people',people
1,b'hello',0.231123,4486,5298655158,hello,b'hello',hello
2,b'people',1.291016,-54580,-1296043274,people,b'people',people
3,b'you',-0.272557,63590,5635528587,you,b'you',you
4,b'people',1.020135,30722,2668741300,people,b'people',people


In [10]:
_.dtypes

bhello      object
f          float64
i32          int32
i64          int64
hello       object
bcat      category
cat       category
dtype: object

As usual, you can use Dask to read the data. This produces a lazy dataframe, and operations on it will work out-of-core, and potentially be distributed across a cluster of workers, if you have them set up. Note that the Dask version has superior metadata inference, so the categorical columns show up even before loading any data.

In [11]:
ddf = source.to_dask()
ddf.dtypes

bhello      object
f          float64
i32          int32
i64          int64
hello       object
bcat      category
cat       category
dtype: object

In [12]:
# direct loading with extra parameters - column selection
from intake_parquet import ParquetSource, Plugin
source = ParquetSource('test.parq', parquet_kwargs={'columns': ['f', 'i32', 'cat']})
source.read().head()

,f,i32,cat
0,1.538369,-68805,people
1,0.231123,4486,hello
2,1.291016,-54580,people
3,-0.272557,63590,you
4,1.020135,30722,people


In [13]:
# same, but using the plugin
Plugin().open('test.parq', columns=['f', 'i32', 'cat']).read()

,f,i32,cat
0,1.538369,-68805,people
1,0.231123,4486,hello
2,1.291016,-54580,people
3,-0.272557,63590,you
4,1.020135,30722,people
5,1.119149,3222,you
6,0.196977,120698,hello
7,-1.668795,-34858,hello
8,1.361207,66829,people
9,1.255644,-75602,hello
